In [1]:
import requests
import time 

authKey = "e0bbaa859b5c43d3887e6cdbabbc2f74"

In [2]:
headers = {
    "authorization": authKey,
    "content-type": "application/json"
}

uploadURL     = "https://api.assemblyai.com/v2/upload"
transcriptURL = "https://api.assemblyai.com/v2/transcript"

In [3]:
def uploadMyFile(filename):
    
    def _readMyFile(fn):
        
        chunk_size = 5242880
        
        with open(fn, "rb") as fileStream:
            
            while True:
                data = fileStream.read(chunk_size)
                if not data:
                    break
                yield data
        pass
    
    response = requests.post(
        uploadURL,
        headers = headers,
        data = _readMyFile(filename)
    )
    
    json = response.json()
    
    return json["upload_url"]



In [4]:
audioURL = uploadMyFile("Recording.mp3")

In [5]:
def startTranscription(aURL):
    
    response = requests.post(
        transcriptURL,
        headers = headers,
        json = {"audio_url" : aURL}
    )
    
    json = response.json()
    
    return json["id"]

In [6]:
transcriptionID = startTranscription(audioURL)

In [9]:
def getTranscription(tId):
    
    maxAttempts = 10
    timedout = False
    
    while True:
        response = requests.get(
            f"{transcriptURL}/{tId}",
            headers = headers,  
        )
        
        json = response.json()
        
        if json["status"] == "completed":
            break
        
        maxAttempts -= 1
        timedout = maxAttempts <= 0
        
        if timedout:
            break
            
        #Waiting 3 seconds before next try
        time.sleep(3)
        
        return TimeoutError if timedout else "completed"


In [10]:
getTranscription(transcriptionID)

In [7]:
def wordSearch(tId, words : list):
    words = ",".join(words)
    response = requests.get(
            f"{transcriptURL}/{tId}/word-search?words={words}",
            headers = headers,  
        )
        
    json = response.json()
    
    return json["matches"]

In [11]:
matches = wordSearch(transcriptionID, ["hello", "dog"])

In [12]:
print(matches)

[{'text': 'hello', 'count': 2, 'timestamps': [[850, 1230], [4430, 5118]], 'indexes': [0, 4]}, {'text': 'dog', 'count': 1, 'timestamps': [[8210, 8960]], 'indexes': [7]}]


In [24]:
lista = [{'text': 'hi', 'count': 2, 'timestamps': [[2750, 3258], [3344, 3980]], 'indexes': [2, 3]}, {'text': 'dog', 'count': 1, 'timestamps': [[8210, 8960]], 'indexes': [7]}]

for item in lista:
    timestamps = []
    item.pop("indexes")
    for time in item["timestamps"]:
        timestamps.append(time[0]/1000)
    item["timestamps"] = ", ".join(str(stamp) for stamp in timestamps)
    
print(lista)

TypeError: sequence item 0: expected str instance, float found